# hdpws HAPI Example Jupyter Notebook
![SPASE inside](https://spase-group.org/assets/images/spase-inside.png)
This [Jupyter notebook](https://jupyter.org/) demonstrates using the [hdpws](https://pypi.org/project/hdpws/) Python package to find [HAPI](http://hapi-server.org/) accessible data from the [Space Physics Archive Search and Extract](https://spase-group.org/) (SPASE) metadata documents at the [Heliophysics Data Portal](https://heliophysicsdata.gsfc.nasa.gov/) (HDP).  It assumes some familarity with the [SPASE data model](https://spase-group.org/data/index.html).  This notebook contains the following sections:
1. [Prerequisites](#Prerequisites)
2. [Setup](#Setup)
3. [Get MeasurementTypes](#Get-MeasurementTypes)
4. [Get ObservedRegions](#Get-ObservedRegions)
5. [Get ObservatoryIDs](#Get-ObservatoryIDs)
6. [Get NumericalData](#Get-NumericalData)
7. [Additional Documentation](#Additional-Documentation)

## Prerequisites
Install the prerequisite software from [Python Package Index](https://pypi.org/project/hdpws/) (PyPI) software repository.
1. pip install hdpws

## Setup
Execute some preliminary code that is necessary before the code that follows.

In [3]:
from hdpws.hdpws import HdpWs
from hdpws import NAMESPACES as NS
from hdpws.resourcetype import ResourceType as rt
from hdpws.spase import HapiAccessURL

from IPython.core.display import HTML

# disable warnings about vspo-dev certificate
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

hdp = HdpWs(endpoint='http://ec2-44-212-210-55.compute-1.amazonaws.com/WS/hdp/1/',
            disable_ssl_certificate_validation=True)

## Get MeasurementTypes
The following code demonstrates how to get the list of available /Spase/MeasurementType values.

In [4]:
result = hdp.get_measurement_types()
print('HDP MeasurementTypes:')
for value in result['MeasurementType']:
    print(f'    {value}')


HDP MeasurementTypes:
    ActivityIndex
    Dopplergram
    Dust
    ElectricField
    EnergeticParticles
    Ephemeris
    ImageIntensity
    InstrumentStatus
    IonComposition
    Irradiance
    MagneticField
    Magnetogram
    NeutralAtomImages
    NeutralGas
    Profile
    Radiance
    Spectrum
    SPICE
    ThermalPlasma
    Waves
    Waves.Active
    Waves.Passive


## Get ObservedRegions
The following code demonstrates how to get the list of available /Spase/ObservedRegion values.

In [5]:
result = hdp.get_observed_regions()
observed_regions = result['ObservedRegion']
print(f'{len(observed_regions)} HDP Observed Regions:')
for value in observed_regions[0:9]:
    print(f'    {value}')
print('    ...')

122 HDP Observed Regions:
    Asteroid
    Comet
    Earth
    Earth.Magnetosheath
    Earth.Magnetosphere
    Earth.Magnetosphere.Magnetotail
    Earth.Magnetosphere.Main
    Earth.Magnetosphere.Plasmasphere
    Earth.Magnetosphere.Polar
    ...


## Get ObservatoryIDs
The following code demonstrates how to get the list of available /Spase/Observatory/ResourceID values.

In [6]:
result = hdp.get_observatory_ids()
observatory_ids = result['ObservatoryID']
print(f'{len(observatory_ids)} HDP ObservatoryIDs:')
for value in observatory_ids[0:9]:
    print(f'    {value}')
print('    ...')

2003 HDP ObservatoryIDs:
    spase://SMWG/Observatory/JSPO
    spase://SMWG/Observatory/MEASURE/Darksky.Observatory
    spase://SMWG/Observatory/THEMIS/Ground/UCLA-EPO/BMLS
    spase://SMWG/Observatory/THEMIS/Ground/UCLA-GBO/GBAY
    spase://SMWG/Observatory/THEMIS/E
    spase://SMWG/Observatory/Ground/Karmoy
    spase://SMWG/Observatory/Ground/Val.Joyeux
    spase://SMWG/Observatory/Ground/Pangnirtung
    spase://SMWG/Observatory/Ground/Kevo
    ...


## Get NumericalData
The following code demonstrates how to find HAPI accessible SPASE NumericalData documents matching the specified search criteria.   

In [7]:
query = {
    'InstrumentID': 'spase://SMWG/Instrument/Wind/MFI',
    'Cadence': '=PT3S',
    'ObservedRegion': 'Heliosphere.NearEarth',
    'MeasurementType': 'MagneticField',
    'AccessRights': 'Open',
    'Style': 'HAPI'
}       
types = [rt.NUMERICAL_DATA]
time_range = ['2022-01-01', '2022-01-02']
result = hdp.get_spase_data(types, query, time_range)
if result['HttpStatus'] == 200:
    for spase in result['Result'].findall('.//Spase',
                                         namespaces=NS):
        id = spase.find('.//ResourceID', namespaces=NS)
        name = spase.find('.//Name', namespaces=NS)
        description = spase.find('.//Description',
                                 namespaces=NS)
        print('Name: ', name.text)
        print('ResourceID: ', id.text)
        print('Description: ', description.text[:60], '...')
        display(HTML('<a href="' + hdp.get_spase_url(id.text) +
                     '" target="_blank">' + 
                    'HTML representation of SPASE</a>'))
        hapi_access_url_element = \
            spase.find('.//AccessURL[Style="HAPI"]', namespaces=NS)
        if hapi_access_url_element is not None:
            hapi_access_url = HapiAccessURL(hapi_access_url_element)
            hapi_display_url = hapi_access_url.get_html_url()
            hapi_html_link = "<a href=""" + \
                hapi_access_url.get_html_url() + \
                """ target=""_blank"">HAPI access information</a>"""
            display(HTML(hapi_html_link))

Name:  Wind Spacecraft Homepage
ResourceID:  spase://NASA/NumericalData/Wind/MFI/PT03S
Description:  Wind MFI composite data in GSE and GSM coordinates. The file ...


## Additional Documentation
View the [hdpws](https://heliophysicsdata.gsfc.nasa.gov/WebServices/py/hdpws/) and [cdasws](https://cdaweb.gsfc.nasa.gov/WebServices/REST/py/cdasws/) API documentation for a description of additional features.